In [1]:
import collections
import pandas as pd
from nltk import agreement
import scipy.stats as stats

#checking for evaulation results
df_labeled = pd.read_csv('../data/evaulation_data/evaluation_set_with_labels_DONT_LOOK.csv')

rating_dfs = {}

for initials in "RZ KK NK".split():
  rating_dfs[initials] = pd.read_csv(f'../data/evaulation_data/Title_Evaluation_Final_{initials}.csv')[['unique_id','Rating']]


df_label_rated = df_labeled.merge(rating_dfs['RZ'], on = 'unique_id').merge(rating_dfs['NK'], on = 'unique_id').merge(rating_dfs['KK'], on = 'unique_id')
df_label_rated['rating_sum'] = df_label_rated[['Rating_x','Rating_y','Rating']].sum(axis = 1)

df_label_rated['rating_percentage'] = df_label_rated['rating_sum']/303 # 300 + 3 ties
df_performance = df_label_rated[['method','rating_sum','rating_percentage']].groupby('method').sum()


lst_spsm = df_label_rated[df_label_rated['method']=='SPSM'][['rating_sum']].rating_sum.tolist()
lst_KNN = df_label_rated[df_label_rated['method']=='KNN'][['rating_sum']].rating_sum.tolist()
lst_naive = df_label_rated[df_label_rated['method']=='naive'][['rating_sum']].rating_sum.tolist()


rater1 = df_label_rated.Rating_x.tolist()
rater2 = df_label_rated.Rating_y.tolist()
rater3 = df_label_rated.Rating.tolist()

taskdata=[[0,str(i),str(rater1[i])] for i in range(0,len(rater1))]+[[1,str(i),str(rater2[i])] for i in range(0,len(rater2))]+[[2,str(i),str(rater3[i])] for i in range(0,len(rater3))]
ratingtask = agreement.AnnotationTask(data=taskdata)


print('Table 1')
print(df_performance)
print()
print('T-Test: naive vs KNN:',stats.ttest_rel(lst_naive, lst_KNN))
print('T-Test: SPSM vs KNN:',stats.ttest_rel(lst_spsm, lst_KNN))
print('-'*100)


print('Table 8')
print(f'kappa {ratingtask.kappa():.4}')
print(f'fleiss {ratingtask.multi_kappa():.4}')
print(f'alpha {ratingtask.alpha():.4}')
print(f'scotts {ratingtask.pi():.4}')


Table 1
        rating_sum  rating_percentage
method                               
KNN            212           0.699670
SPSM            53           0.174917
naive           38           0.125413

T-Test: naive vs KNN: TtestResult(statistic=-10.752658157882099, pvalue=2.5121072302919953e-18, df=99)
T-Test: SPSM vs KNN: TtestResult(statistic=-8.73277055548224, pvalue=6.306465360934115e-14, df=99)
----------------------------------------------------------------------------------------------------
Table 8
kappa 0.5621
fleiss 0.5622
alpha 0.5627
scotts 0.5622
